# NResumes

### Restrictions
The subheadings need to contain just the text without any embellishments  
The date needs to be in the format January/Feb 2019- February/Feb 2020

In [92]:
from PyPDF2 import PdfFileReader
import os
from os import listdir
from os.path import isfile, join
import re
import nltk
from pyresparser import ResumeParser
import pandas as pd
import operator
import shutil

## The paths for the Resume and CSV files

In [93]:
resume_path = 'resume' # Should be decided by the front end
profile_path = "csv"

In [94]:
job_req = {'Data Science': {'Python':['scikitlearn','pandas','matplotlib'], 'Machine Learning':['Regression','ML','KNN'], 'Querying':['SQL','Hive','Pig'], 'Database':['SQL','MongoDB','Cassandra']},
           "Data Engineer": {'Language':['Java','Scala','Python']},
           "Front-End": {'Website':['HTML','CSS','JavaScript','JS'], 'Mobile':['React','React-Native','Flutter','Android','iOS']},
           "Back-End": {'DBMS':['MySQL', 'MongoDB', 'Oracle', 'SQLServer', 'Redis'], 'Language':['Java', 'Python', 'Ruby', '.net']},
           "Software Engineer": {'DBMS':['MySQL', 'MongoDB', 'Oracle', 'SQLServer', 'Redis'], 'Language':['Java', 'Python', 'Ruby', '.net']}
          }

## Reading in CSV data into a DataFrame  (Try to autocreate the file directory for each job)

In [95]:
profile_data = {} # a dictionary to store all the dataframes

#get a list consisting of the paths of all the csv files 
profile_files = [os.path.join(profile_path, f) for f in os.listdir(profile_path) if os.path.isfile(os.path.join(profile_path, f))]

for file in profile_files:
    keyname = os.path.basename(file)[:-len('.csv')]  #this is just to obtain every file name sans the extension  
    #print(keyname)
    x = pd.read_csv(file)
    profile_data[keyname] = x.apply(lambda x:x.astype(str).str.lower()) 

profiles_list = list(profile_data.keys()) #Just storing the profile in list format


#type(profile_data['data_scientist'].columns[0])

profiles_list   
#profile_data['data_scientist']# printing a sample one for data scientist
#profile_data['full_stack_dev']# printing a sample one for data scientist
#profile_data['cloud_architect']# printing a sample one for data scientist

['business_analyst',
 'cloud_architect',
 'data_scientist',
 'full_stack_dev',
 'testing']

## Classifying resumes on job description and storing them in their respective folders
We have a dataframe that has the skills for each job, we shall iterate through the folder containing all resumes, for each resume calculate the skills (add +1 for each mention of the skill) and store it in the job folder that has the max score. This is so that we can display his proficiency in not only all the skills mentioned but also other skills that the job entails

### Since we are parsing through the requirements anyways, create a list of dictionary of scores each of the skills in each job and then assign the most proficient job's dictionary to the final list of dictionary.

In [155]:
#for each file in resume_path, reading the resumes and extracting info

skills_list = [{} for i in profiles_list]  # a dictionary of skills for representing the skills of employee in each job

employee_data = [] #a list of dicts representing the employee's data along with his skill metric

for f in os.listdir(resume_path):

    if os.path.isfile(os.path.join(resume_path, f)):
        
        data = ResumeParser(os.path.join(resume_path, f)).get_extracted_data()
        print(data)
        print(" ")
    
        proficiency = []
        
        #Iterating through all the job dataframes in profile_data and 
        #For each job, iterate through all skills 
        #If skills match skills from resume increment specific_skills by one
        
        skills_lower = [i.lower() for i in data['skills']] #convert all the skills to lowercase
        
        for job_no,tup in enumerate(profile_data.items()):
            
            jobs,jobs_df = tup 
            
            talent_in_job=0  #a metric to measure skill in a job profile based on number of mentions in resume
            
            for skills_col in jobs_df.columns:
                #print(jobs_df[skills_col].iloc[0])
                #skills_col_lower=jobs_df[skills_col].str.lower()
                #print(len(list(set(data_series).intersection(set(skills_col_lower)))))
                
                specific_skill_level = len(list(set(skills_lower).intersection(set(jobs_df[skills_col]))))
                
                skills_list[job_no][skills_col] = specific_skill_level
                
                talent_in_job+=specific_skill_level
                
            
            proficiency.append(talent_in_job)
            #proficiency_measure[jobs] = talent_in_job                
        
        #get the name of the job that has most matches and move the file to the job's folder
        
        most_proficient = proficiency.index(max(proficiency))

        data['skills'] = skills_lower
        data['proficiency'] = profiles_list[most_proficient]
        data['job_skills_dict'] = skills_list[most_proficient]
        
        employee_data.append(data)
        new_directory = os.path.join(resume_path,profiles_list[most_proficient],f)
        old_directory = os.path.join(resume_path, f)
        shutil.move(old_directory,new_directory)
        
print(employee_data)

{'name': 'JOSHI OBJECTIVE', 'email': 'vgjoshi2297@gmail.com', 'mobile_number': '9175300022', 'skills': ['Hardware', 'Matlab', 'Communication', 'R', 'Instrumentation', 'Training', 'Research', 'Twitter', 'Electrical', 'Electronics'], 'college_name': None, 'degree': ['B. Tech in Electronics and Instrumentation'], 'designation': None, 'experience': ['\uf0b7  Currently working on a research project', 'involving visible light communication and', 'li-fi.', 'ACOMPLISHMENTS', '\uf0b7  Achieved scholarship in VIT University:  25% scholarship in VITEEE Exam.', '\uf0b7  Certified in Labview training. Link to the certificate:', 'https://drive.google.com/file/d/1yw-', 'Q936UJFQkYaB2PQOY7le2JhYyQ0nu/view?usp=sharing'], 'company_names': None, 'no_of_pages': 1, 'total_experience': 0.0}
 
{'name': 'ADITYA CAMARUSHY', 'email': 'aditcam@gmail.com', 'mobile_number': '8971198767', 'skills': ['Testing', 'Big data', 'Octave', 'Css', 'Javascript', 'Health', 'Software engineering', 'Operating systems', 'Technic

## Taking Input for skills, experience and how many resumes

In [156]:
job_no=0
for job_roles in profiles_list:
    print(f'{job_no+1}.) {job_roles}')
    job_no+=1

1.) business_analyst
2.) cloud_architect
3.) data_scientist
4.) full_stack_dev
5.) testing


In [157]:
user_requirements = []
requirement = None

    
desired_job = int(input("What's the desired no. of the job profile? "))-1

while requirement != 'q':
    requirement = input("What are the skills needed? (q to exit)")
    if requirement != 'q':
        user_requirements.append(requirement.lower())
    
num_resumes = int(input("How many resumes would you like identified?"))

yrs_exp = int(input("How many years of experience is the candidate supposed to have?"))

What's the desired no. of the job profile? 4
What are the skills needed? (q to exit)html
What are the skills needed? (q to exit)css
What are the skills needed? (q to exit)q
How many resumes would you like identified?2
How many years of experience is the candidate supposed to have?1


### Creating a function that ranks people based on experience and skills

In [158]:
print(user_requirements)

['html', 'css']


In [162]:
def ranker():
    job = profiles_list[desired_job]
    #print(job)
    
    x = [i for i in employee_data if i['proficiency']==job]
    #print(x)
    
    for job_dicts in x:
        #print(job_dicts['skills'])
        exp = job_dicts['total_experience']
        exp_score = 50*(exp/yrs_exp)
        #print(set(user_requirements).intersection(set(job_dicts['skills'])))
        req_met = len(list(set(user_requirements).intersection(set(job_dicts['skills']))))
        req_score = 50*(req_met/len(user_requirements))
        job_dicts['score'] = exp_score+req_score
        
    x = sorted(x,key=lambda i:i['score'],reverse=True)
    
    return(x)

In [163]:
ranker()

[{'name': 'ADITYA CAMARUSHY',
  'email': 'aditcam@gmail.com',
  'mobile_number': '8971198767',
  'skills': ['testing',
   'big data',
   'octave',
   'css',
   'javascript',
   'health',
   'software engineering',
   'operating systems',
   'technical',
   'programming',
   'java',
   'linux',
   'sql',
   'matlab',
   'parser',
   'analytics',
   'mobile',
   'analysis',
   'windows',
   'engineering',
   'python',
   'billing',
   'photography',
   'c',
   'html',
   'data analytics',
   'email'],
  'college_name': None,
  'degree': ['B.Tech'],
  'designation': None,
  'experience': None,
  'company_names': None,
  'no_of_pages': 2,
  'total_experience': 0,
  'proficiency': 'full_stack_dev',
  'job_skills_dict': {'Programming Languages': 2,
   'Operating Systems': 0,
   'Backend': 1,
   'Technologies': 0,
   'Web Frameworks': 0},
  'score': 50.0},
 {'name': 'JOSHI OBJECTIVE',
  'email': 'vgjoshi2297@gmail.com',
  'mobile_number': '9175300022',
  'skills': ['hardware',
   'matlab',
  